In [2]:

# !conda install -c conda-forge folium=0.5.0 --yes
# !conda install -c conda-forge geopy --yes
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows',500)
import json
import requests #
from pandas import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium # map rendering library
# !pip install lxml
import lxml
import time
print('Done!')

Done!


### Get Data from WiKi

In [3]:
# Get data from wiki
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url)

In [6]:
df_temp= pd.DataFrame(data[0])
df_temp.dropna(axis=0, inplace=True) # Drop all NaN values from dataframe's rows - it's empty Neighborhood and 'Not assigned'Borough
df_temp.reset_index(inplace=True) # reset index
df_temp.drop(columns='index',inplace=True) # delete old index column
df_temp.Neighborhood=[x.split(',') for x in df_temp.Neighborhood] # change values in Neighborhood in array
column_names = ['PostalCode','Borough', 'Neighborhood'] # prepare new dataframe
df  = pd.DataFrame(columns=column_names) # new dataframe
df_temp.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]"
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]"
4,M7A,Downtown Toronto,"[Queen's Park, Ontario Provincial Government]"


In [7]:
df

,PostalCode,Borough,Neighborhood


In [8]:
ts=time.time()
for index, x in enumerate(df_temp.Neighborhood): # filll new dataframe values. One Neighborhood - one row 
    for neigh in x:
        df = df.append ({'PostalCode':df_temp['Postal Code'][index],
                         'Borough':df_temp.Borough[index],
                         'Neighborhood':neigh },ignore_index=True)
df=df[column_names]
print(df.head())
print('Shape: {}\t\t\tTime: {:.2f}' .format(df.shape, time.time()-ts))

   PostalCode           Borough                    Neighborhood
0         M3A        North York                       Parkwoods
1         M4A        North York                Victoria Village
2         M5A  Downtown Toronto                     Regent Park
3         M5A  Downtown Toronto                    Harbourfront
4         M6A        North York                  Lawrence Manor
5         M6A        North York                Lawrence Heights
6         M7A  Downtown Toronto                    Queen's Park
7         M7A  Downtown Toronto   Ontario Provincial Government
8         M9A         Etobicoke                Islington Avenue
9         M1B       Scarborough                         Malvern
10        M1B       Scarborough                           Rouge
11        M3B        North York                       Don Mills
12        M4B         East York                   Parkview Hill
13        M4B         East York                Woodbine Gardens
14        M5B  Downtown Toronto         

### Get Data Location from WiKi